# RoBERTa Fine-tuning

## 1. import

In [2]:
import os
import sys

# TODO: change this to the path to your homework folder
GOOGLE_DRIVE_PATH = os.path.join('drive', 'MyDrive')
print(os.listdir(GOOGLE_DRIVE_PATH))

os.chdir(GOOGLE_DRIVE_PATH)

['4', 'Colab Notebooks', 'df_test_pickle', 'df_train_pickle', 'df_val_pickle', 'image', 'text_embedding.py', 'result.pt', 'text', 'val_embedding_VGG_finetuned.pt', 'image_embedding_VGG_finetuned_enhanced.pt', 'test_gold.csv', 'test_embedding_VGG_finetuned.pt', '.ipynb_checkpoints', 'train_text_embedding_R.pt', 'val_text_embedding_R.pt', 'test_text_embedding_R.pt', 'model.pt', 'train_text_embed_ft', 'val_text_embed_ft', 'test_text_embed_ft', 'meme.ipynb']


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.1 MB/s eta 0:00:00


In [5]:
# import packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch

# import directories
import image
from text import text_embedding
import nltk
nltk.download('punkt')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


## 2. Load dataset

In [6]:
# load labels_pd_pickle
train = pd.read_pickle('df_train_pickle')
val = pd.read_pickle('df_val_pickle')
test = pd.read_pickle('df_test_pickle')
# show first 5 rows
val['sarcastic']

0          little_sarcastic
1          little_sarcastic
2             not_sarcastic
3            very_sarcastic
4             not_sarcastic
               ...         
1495       little_sarcastic
1496       little_sarcastic
1497    extremely_sarcastic
1498       little_sarcastic
1499          not_sarcastic
Name: sarcastic, Length: 1500, dtype: object

In [9]:
def basic_collate_fn_all(batch):
    """Collate function for basic setting."""
    # texts
    text = torch.stack(tuple(item['text'] for item in batch)).to('cuda')
    # image
    image = torch.stack(tuple(item['image_embedding'] for item in batch)).to('cuda')
    # labels
    labels = [item['rating'][0:4] for item in batch]
    return text, image, labels

# 3. Fine tuning

In [ ]:
import nltk
nltk.download('punkt')
def basic_collate_fn(batch):
    """Collate function for basic setting."""
    # texts
    text = [item['text'] for item in batch]
    # labels
    labels = [item['rating'] for item in batch]
    return text, labels

In [ ]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
class dataloader_ft(Dataset):
    """Generate data loader."""
    def __init__(self, trainset):
        super().__init__()
        self.items = []
        for index, row in trainset.iterrows():
            self.items.append({
                'text': row['ocr_text'],
                'rating': [text_embedding.ambiguity(row['ocr_text']),text_embedding.incongruity(row['ocr_text'])]
            })
    
    def __len__(self):
        return len(self.items)
    
    def __getitem__(self, index):
        return self.items[index]
train_ft_data = dataloader_ft(train)
train_ft_loader = DataLoader(train_ft_data[0:1500], batch_size=32, collate_fn=basic_collate_fn, shuffle=True)

In [ ]:
def _train(model,train_loader, optimizer):
    model.train()
    for epoch in range(1, 5):
        for batch_idx, (data, target) in enumerate(train_loader):
            optimizer.zero_grad()
            output = model.forward(data)
            label = torch.Tensor(target).to('cuda')
            loss = get_loss(output, label)
            loss.backward()
            optimizer.step()
            if batch_idx % 100 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset), 100. * batch_idx / len(train_loader), loss.item()))
    return model

In [ ]:
def get_lossfunction():
    """Get loss function."""
    return nn.MSELoss()

def get_loss(output, target):
    """Get loss."""
    func = get_lossfunction()
    return func(output, target)

In [ ]:
from transformers import RobertaTokenizer, RobertaModel
from torch import nn
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
class Finetuner(nn.Module):
    def __init__(self):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained(model_name).to('cuda')
        self.linear = nn.Linear(768, 2)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = text_embedding.text_tokened(x).to('cuda')
        output = self.roberta(**x)
        x = output.last_hidden_state[:, -1, :]
        x = self.linear(x)
        x = self.sigmoid(x)
        return x.squeeze()


In [ ]:
import torch.optim as optim
from torch import nn
model = Finetuner().to('cuda')
optimizer = optim.Adam(model.parameters(),lr=0.000001)
model = _train(model,train_ft_loader,optimizer)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Train Epoch: 1 [0/1500 (0%)]	Loss: 0.272351
Train Epoch: 2 [0/1500 (0%)]	Loss: 0.194018
Train Epoch: 3 [0/1500 (0%)]	Loss: 0.043893
Train Epoch: 4 [0/1500 (0%)]	Loss: 0.027891


In [ ]:
torch.save(model,'model.pt')

In [ ]:
for item in train['ocr_text'][0:2]:
  item = text_embedding.text_tokened(item).to('cuda')
  output = model.roberta(**item)
  x = output.last_hidden_state[:, -1, :].squeeze()
  print(x[0:10])

tensor([-0.8089,  0.2790, -0.6024,  0.0591, -0.4000,  0.5021, -0.1828, -0.1156,
        -0.4835, -0.5040], device='cuda:0', grad_fn=<SliceBackward0>)
tensor([-0.3450, -0.3232, -0.5046, -0.0215,  0.2475, -0.1267, -0.0947, -0.2307,
        -1.0065, -0.4971], device='cuda:0', grad_fn=<SliceBackward0>)


In [ ]:
model.to('cpu')
del embed
embed = torch.load('test_text_embed_ft')
index = 1000
for item in test['ocr_text'][index:index+500]: # in case of RAM overflow
  item = text_embedding.text_tokened(item)
  output = model.roberta(**item)
  x = output.last_hidden_state[:, -1, :].squeeze()
  embed.append(x)
  del item
  del x
  del output
print(len(embed))
torch.save(embed,'test_text_embed_ft')

1500
